# Function Call - Google Custom Search API 版

# 準備

In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build

# 環境変数の取得
load_dotenv()
# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
model_name = "gpt-3.5-turbo-1106"

google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']

# Google検索API

In [3]:
import json

# ニュースを返す関数の作成
def get_search_result(keyword, when, unit="d"):
    service = build("customsearch", "v1", developerKey=google_api_key)
    result = service.cse().list(q=keyword, dateRestrict=f"{when}{unit}", cx=google_cse_id).execute()

    # 検索結果のトップ3を配列に取得
    result_list = []
    for i, item in enumerate(result["items"]):
        result_list.append({"title": f'{item["title"]} {item["snippet"]}'})
        if i>=2:
            break

    return json.dumps({"result": result_list})

In [4]:
# テスト用コード
ret = get_search_result("東京駅のイベント", 1, "m")
ret2 = json.loads(ret)
for n in ret2["result"]:
    print("-"*20)
    print(n["title"])

--------------------
東京駅周辺・丸の内でおすすめのイベント｜レッツエンジョイ東京 東京駅周辺・丸の内でおすすめのイベント · かごしま遊楽館 / 千代田区立日比谷図書文化館 / 都立東綾瀬公園 / 楠木正成像 / 綾瀬駅（東京メトロ） / 日比谷駅（東京メトロ ...
--------------------
東京キャラクターストリート特設 ｜ 東京駅一番街 東京キャラクターストリート内の「いちばんプラザ」やワゴンショップなど期間限定ショップやイベントも開催中！ いつも楽しいワクワクを集めてみなさまのお越しを ...
--------------------
東京駅(東京都)周辺のイベント｜ウォーカープラス 東京駅(東京都)周辺で開催されるイベント情報9件をお届けします。今日開催されているイベントはもちろん、週末の「どこ行こう」に役立つ情報が満載！


In [5]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [6]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

In [7]:
response

ChatCompletion(id='chatcmpl-8eFyk8dTZMTe5nKRQInFaO1lfkSgW', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_d6wBsnCxvQP412p0GZf5fweu', function=Function(arguments='{"keyword":"東京駅 イベント","when":1,"unit":"m"}', name='get_search_result'), type='function')]))], created=1704605234, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_cbe4fa03fe', usage=CompletionUsage(completion_tokens=33, prompt_tokens=135, total_tokens=168))

In [8]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"東京駅 イベント","when":1,"unit":"m"}'

In [7]:
# モデルがツール呼出と判断した
if response.choices[0].message.tool_calls is not None:
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0].message.content.strip())

関数名：get_search_result
引数：{'keyword': '東京駅 イベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "\u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u30f3\u30c8\uff5c\u30ec\u30c3\u30c4\u30a8\u30f3\u30b8\u30e7\u30a4\u6771\u4eac \u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u30f3\u30c8 \u00b7 \u304b\u3054\u3057\u307e\u904a\u697d\u9928 / \u5343\u4ee3\u7530\u533a\u7acb\u65e5\u6bd4\u8c37\u56f3\u66f8\u6587\u5316\u9928 / \u90fd\u7acb\u6771\u7dbe\u702c\u516c\u5712 / \u6960\u6728\u6b63\u6210\u50cf / \u7dbe\u702c\u99c5\uff08\u6771\u4eac\u30e1\u30c8\u30ed\uff09 / \u65e5\u6bd4\u8c37\u99c5\uff08\u6771\u4eac\u30e1\u30c8\u30ed\u00a0..."}, {"title": "\u30a4\u30d9\u30f3\u30c8&\u30cb\u30e5\u30fc\u30b9 \uff5c \u6771\u4eac\u99c5\u4e00\u756a\u8857 \u30a4\u30d9\u30f3\u30c8&\u30cb\u30e5\u30fc\u30b9 \u00b7 New! \u30a6\u30eb\u30c8\u30e9\u30de\u30f3\u30ef\u30fc\u30eb\u30c9M78 \u30b9\u30da\u30b7\u30

# 複数のFunctionの切り替え

In [8]:
import json
import requests
from bs4 import BeautifulSoup

# ニュースを検索する関数
def get_news(keyword, when, unit="d"):
    # WebサイトのURLを指定
    url = f'https://news.google.com/search?q={keyword} when:{when}{unit}&hl=ja&gl=JP&ceid=JP:ja'

    # Requestsを利用してWebページを取得する
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div div div a')

    # ニュースのトップ3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i>=2:
            break

    return json.dumps({"news": news})

In [9]:
# テスト用コード
ret = get_news("お祭り", 5)
ret2 = json.loads(ret)
for n in ret2["news"]:
    print("-"*20)
    print(n["title"])

--------------------
「Happy Holi！」NYに春の訪れを告げるお祭り
--------------------
祭りで企業や商品をPR｜株式会社オマツリジャパン
--------------------
ふれあい中央 カーニバル（神戸市中央区のお祭り） - はぎわら泰三（ハギワラタイゾウ） ｜ 選挙ドットコム


In [10]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    }),
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_news",
            "description": "指定したキーワードのニュースを取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [12]:
# question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
question = "お祭りに関する5日以内のニュースを教えてください"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)
response

ChatCompletion(id='chatcmpl-9GyoKYwwKiOqjJfoRCA6Ji7pfGaeH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CQhEIvDHXxffGgzDF8fObvLy', function=Function(arguments='{"keyword":"お祭り","when":5,"unit":"d"}', name='get_news'), type='function')]))], created=1713834032, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_77a673219d', usage=CompletionUsage(completion_tokens=25, prompt_tokens=187, total_tokens=212))

In [22]:
response

ChatCompletion(id='chatcmpl-8eGMPkDTObpmHLxFg4GXO5jLU9X0n', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_isGvMM3hSd4SIR445nLLPmyf', function=Function(arguments='{"keyword":"お祭り","when":5,"unit":"d"}', name='get_news'), type='function')]))], created=1704606701, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_cbe4fa03fe', usage=CompletionUsage(completion_tokens=25, prompt_tokens=187, total_tokens=212))

In [23]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"お祭り","when":5,"unit":"d"}'

In [14]:
# モデルがツール呼出と判断した
if response.choices[0].message.tool_calls is not None:
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    # 引数の取得
    arguments = json.loads(tool.function.arguments)

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    elif function_name == "get_news":
        function_response = get_news(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0].message.content.strip())

最新のお祭りに関するニュースは以下の通りです。

1. 「Happy Holi！」NYに春の訪れを告げるお祭り
2. お祭りで企業や商品をPR｜株式会社オマツリジャパン
3. 仏街中央 カーニバル（神戸市中央区のお祭り） - はぎわ和三（ハギワラタイゾウ）｜選挙ドットコム
